# 01 Data Prep
Prepares results of NCAA tournament

In [1]:
import pandas as pd
import numpy as np

## Read in Data

In [2]:
seasons = pd.read_csv('../data/Stage2DataFiles/seasons.csv')
seasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


In [3]:
results_compact = pd.read_csv('../data/Stage2DataFiles/NCAATourneyCompactResults.csv')
results_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [4]:
teams = pd.read_csv('../data/Stage2DataFiles/teams.csv')
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2019
1,1102,Air Force,1985,2019
2,1103,Akron,1985,2019
3,1104,Alabama,1985,2019
4,1105,Alabama A&M,2000,2019


In [5]:
tourney_slots = pd.read_csv('../data/Stage2DataFiles/NCAATourneySlots.csv')
tourney_slots.head()

,Season,Slot,StrongSeed,WeakSeed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12


In [6]:
tourney_seeds = pd.read_csv('../data/Stage2DataFiles/NCAATourneySeeds.csv')
tourney_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [7]:
tourney_seed_round = pd.read_csv('../data/Stage2DataFiles/NCAATourneySeedRoundSlots.csv')
tourney_seed_round.head()

,Seed,GameRound,GameSlot,EarlyDayNum,LateDayNum
0,W01,1,R1W1,136,137
1,W01,2,R2W1,138,139
2,W01,3,R3W1,143,144
3,W01,4,R4W1,145,146
4,W01,5,R5WX,152,152


## Data Prep

In [8]:
# Eliminate 2019 season and add round variable
tourney_slots_v2 = tourney_slots.copy()

print (len(tourney_slots_v2))
tourney_slots_v2 = tourney_slots_v2.loc[tourney_slots_v2['Season']!=2019]
print (len(tourney_slots_v2))

@np.vectorize
def add_round(slot):
    if slot[0:1]!='R':
        return 0
    else:
        return int(slot.split('R')[1][0])

tourney_slots_v2['Round'] = add_round(tourney_slots_v2['Slot'])
display(tourney_slots_v2['Round'].value_counts())
tourney_slots_v2.head()

2251
2184


1    1088
2     544
3     272
4     136
5      68
0      42
6      34
Name: Round, dtype: int64

,Season,Slot,StrongSeed,WeakSeed,Round
0,1985,R1W1,W01,W16,1
1,1985,R1W2,W02,W15,1
2,1985,R1W3,W03,W14,1
3,1985,R1W4,W04,W13,1
4,1985,R1W5,W05,W12,1


In [9]:
# convert results to season, round, team format
tourney_seed_early_day = tourney_seed_round.groupby('GameRound').agg({'EarlyDayNum':'mean'}).reset_index()
tourney_seed_late_day = tourney_seed_round.groupby('GameRound').agg({'LateDayNum':'mean'}).reset_index()


results_round = pd.merge(results_compact, tourney_seed_early_day, 
                         left_on=['DayNum'], right_on=['EarlyDayNum'], 
                         how='left', 
                         validate='m:1')

results_round = pd.merge(results_round, tourney_seed_late_day, 
                         left_on=['DayNum'], right_on=['LateDayNum'], 
                         how='left', 
                         validate='m:1')

assert len(results_round.loc[
    (pd.isnull(results_round['GameRound_x'])) & 
    (pd.isnull(results_round['GameRound_y']))]
          )==0

valids = results_round.loc[
    (~pd.isnull(results_round['GameRound_x'])) & 
    (~pd.isnull(results_round['GameRound_y']))]
assert len(valids.loc[valids['GameRound_x']!=valids['GameRound_y']])==0

results_round['Round'] = results_round['GameRound_x'].combine_first(results_round['GameRound_y'])

results_w = results_round[['Season', 'Round', 'WTeamID', 'WScore', 'NumOT']].rename(columns={'WTeamID':'TeamID', 'WScore':'Score'})
results_l = results_round[['Season', 'Round', 'LTeamID', 'LScore', 'NumOT']].rename(columns={'LTeamID':'TeamID', 'LScore':'Score'})

results = results_w.append(results_l).reset_index(drop=True)
results.head()

,Season,Round,TeamID,Score,NumOT
0,1985,1.0,1116,63,0
1,1985,1.0,1120,59,0
2,1985,1.0,1207,68,0
3,1985,1.0,1229,58,0
4,1985,1.0,1242,49,0


## Define functions

In [10]:
def create_round_results(seeds, rnd, teams, slots, res):
    seed_info = pd.merge(seeds, teams[['TeamID', 'TeamName']], on=['TeamID'], validate='m:1')
    
    # merge seed info with slots
    r_slots = slots.loc[slots['Round']==rnd]

    r_slot_seed = pd.merge(r_slots, seed_info,
                            left_on=['Season', 'StrongSeed'], right_on=['Season', 'Seed'], 
                            validate='1:1')
    r_slot_seed.drop('Seed', axis=1, inplace=True)

    r_slot_seed = pd.merge(r_slot_seed, seed_info, 
                            left_on=['Season', 'WeakSeed'], right_on=['Season', 'Seed'], 
                            validate='1:1', 
                            suffixes=['_Strong', '_Weak'])
    r_slot_seed.drop('Seed', axis=1, inplace=True)
    
    # add results
    r_results = pd.merge(r_slot_seed, res, 
                          left_on=['Season', 'Round', 'TeamID_Strong'], 
                          right_on=['Season', 'Round', 'TeamID'], 
                          validate='1:1')
    r_results.drop('TeamID', axis=1, inplace=True)

    r_results = pd.merge(r_results, res, 
                          left_on=['Season', 'Round', 'TeamID_Weak'], 
                          right_on=['Season', 'Round', 'TeamID'], 
                          validate='1:1', 
                          suffixes=['_Strong', '_Weak'])
    r_results.drop('TeamID', axis=1, inplace=True)


    r_results['TeamID_Winner'] = [w if ws>ss else s for w,s,ws,ss in 
                                   zip(r_results['TeamID_Weak'], r_results['TeamID_Strong'], 
                                       r_results['Score_Weak'], r_results['Score_Strong'])]
    return r_results

## Round 0

In [11]:
tourney_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [12]:
r0_results = create_round_results(tourney_seeds, 0, teams, tourney_slots_v2, results)
r0_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,Score_Strong,NumOT_Strong,Score_Weak,NumOT_Weak,TeamID_Winner
0,2001,Y16,Y16a,Y16b,0,1322,Northwestern LA,1457,Winthrop,71,0,67,0,1322
1,2002,W16,W16a,W16b,0,1108,Alcorn St,1373,Siena,77,0,81,0,1373
2,2003,X16,X16a,X16b,0,1411,TX Southern,1421,UNC Asheville,84,1,92,1,1421
3,2004,Z16,Z16a,Z16b,0,1197,Florida A&M,1250,Lehigh,72,0,57,0,1197
4,2005,Z16,Z16a,Z16b,0,1105,Alabama A&M,1324,Oakland,69,0,79,0,1324


## Round 1

In [13]:
r1_seeds = r0_results[['Season', 'TeamID_Winner', 'Slot']].copy()
r1_seeds.rename(columns={'TeamID_Winner':'TeamID', 'Slot':'Seed'}, inplace=True)

r1_seeds = r1_seeds.append(tourney_seeds).reset_index(drop=True)

r1_seeds.head()

/Users/matt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Season,Seed,TeamID
0,2001,Y16,1322
1,2002,W16,1373
2,2003,X16,1421
3,2004,Z16,1197
4,2005,Z16,1324


In [14]:
r1_results = create_round_results(r1_seeds, 1, teams, tourney_slots_v2, results)
r1_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,Score_Strong,NumOT_Strong,Score_Weak,NumOT_Weak,TeamID_Winner
0,1985,R1W1,W01,W16,1,1207,Georgetown,1250,Lehigh,68,0,43,0,1207
1,1985,R1W2,W02,W15,1,1210,Georgia Tech,1273,Mercer,65,0,58,0,1210
2,1985,R1W3,W03,W14,1,1228,Illinois,1318,Northeastern,76,0,57,0,1228
3,1985,R1W4,W04,W13,1,1260,Loyola-Chicago,1233,Iona,59,0,58,0,1260
4,1985,R1W5,W05,W12,1,1374,SMU,1330,Old Dominion,85,0,68,0,1374


## Round 2

In [15]:
r2_seeds = r1_results[['Season', 'TeamID_Winner', 'Slot']].copy()
r2_seeds.rename(columns={'TeamID_Winner':'TeamID', 'Slot':'Seed'}, inplace=True)
r2_seeds.head()

,Season,TeamID,Seed
0,1985,1207,R1W1
1,1985,1210,R1W2
2,1985,1228,R1W3
3,1985,1260,R1W4
4,1985,1374,R1W5


In [16]:
r2_results = create_round_results(r2_seeds, 2, teams, tourney_slots_v2, results)
r2_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,Score_Strong,NumOT_Strong,Score_Weak,NumOT_Weak,TeamID_Winner
0,1985,R2W1,R1W1,R1W8,2,1207,Georgetown,1396,Temple,63,0,46,0,1207
1,1985,R2W2,R1W2,R1W7,2,1210,Georgia Tech,1393,Syracuse,70,0,53,0,1210
2,1985,R2W3,R1W3,R1W6,2,1228,Illinois,1208,Georgia,74,0,58,0,1228
3,1985,R2W4,R1W4,R1W5,2,1260,Loyola-Chicago,1374,SMU,70,0,57,0,1260
4,1985,R2X1,R1X1,R1X8,2,1385,St John's,1116,Arkansas,68,0,65,0,1385


## Round 3

In [17]:
r3_seeds = r2_results[['Season', 'TeamID_Winner', 'Slot']].copy()
r3_seeds.rename(columns={'TeamID_Winner':'TeamID', 'Slot':'Seed'}, inplace=True)
r3_seeds.head()

,Season,TeamID,Seed
0,1985,1207,R2W1
1,1985,1210,R2W2
2,1985,1228,R2W3
3,1985,1260,R2W4
4,1985,1385,R2X1


In [18]:
r3_results = create_round_results(r3_seeds, 3, teams, tourney_slots_v2, results)
r3_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,Score_Strong,NumOT_Strong,Score_Weak,NumOT_Weak,TeamID_Winner
0,1985,R3W1,R2W1,R2W4,3,1207,Georgetown,1260,Loyola-Chicago,65,0,53,0,1207
1,1985,R3W2,R2W2,R2W3,3,1210,Georgia Tech,1228,Illinois,61,0,53,0,1210
2,1985,R3X1,R2X1,R2X4,3,1385,St John's,1246,Kentucky,86,0,70,0,1385
3,1985,R3X2,R2X2,R2X3,3,1104,Alabama,1301,NC State,55,0,61,0,1301
4,1985,R3Y1,R2Y1,R2Y4,3,1328,Oklahoma,1256,Louisiana Tech,86,1,84,1,1328


## Round 4

In [19]:
r4_seeds = r3_results[['Season', 'TeamID_Winner', 'Slot']].copy()
r4_seeds.rename(columns={'TeamID_Winner':'TeamID', 'Slot':'Seed'}, inplace=True)
r4_seeds.head()

,Season,TeamID,Seed
0,1985,1207,R3W1
1,1985,1210,R3W2
2,1985,1385,R3X1
3,1985,1301,R3X2
4,1985,1328,R3Y1


In [20]:
r4_results = create_round_results(r4_seeds, 4, teams, tourney_slots_v2, results)
r4_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,Score_Strong,NumOT_Strong,Score_Weak,NumOT_Weak,TeamID_Winner
0,1985,R4W1,R3W1,R3W2,4,1207,Georgetown,1210,Georgia Tech,60,0,54,0,1207
1,1985,R4X1,R3X1,R3X2,4,1385,St John's,1301,NC State,69,0,60,0,1385
2,1985,R4Y1,R3Y1,R3Y2,4,1328,Oklahoma,1272,Memphis,61,0,63,0,1272
3,1985,R4Z1,R3Z1,R3Z2,4,1437,Villanova,1314,North Carolina,56,0,44,0,1437
4,1986,R4W1,R3W1,R3W2,4,1181,Duke,1298,Navy,71,0,50,0,1181


## Round 5

In [21]:
r5_seeds = r4_results[['Season', 'TeamID_Winner', 'Slot']].copy()
r5_seeds.rename(columns={'TeamID_Winner':'TeamID', 'Slot':'Seed'}, inplace=True)
r5_seeds.head()

,Season,TeamID,Seed
0,1985,1207,R4W1
1,1985,1385,R4X1
2,1985,1272,R4Y1
3,1985,1437,R4Z1
4,1986,1181,R4W1


In [22]:
r5_results = create_round_results(r5_seeds, 5, teams, tourney_slots_v2, results)
r5_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,Score_Strong,NumOT_Strong,Score_Weak,NumOT_Weak,TeamID_Winner
0,1985,R5WX,R4W1,R4X1,5,1207,Georgetown,1385,St John's,77,0,59,0,1207
1,1985,R5YZ,R4Y1,R4Z1,5,1272,Memphis,1437,Villanova,45,0,52,0,1437
2,1986,R5WX,R4W1,R4X1,5,1181,Duke,1242,Kansas,71,0,67,0,1181
3,1986,R5YZ,R4Y1,R4Z1,5,1261,LSU,1257,Louisville,77,0,88,0,1257
4,1987,R5WX,R4W1,R4X1,5,1393,Syracuse,1344,Providence,77,0,63,0,1393


## Round 6

In [23]:
r6_seeds = r5_results[['Season', 'TeamID_Winner', 'Slot']].copy()
r6_seeds.rename(columns={'TeamID_Winner':'TeamID', 'Slot':'Seed'}, inplace=True)
r6_seeds.head()

,Season,TeamID,Seed
0,1985,1207,R5WX
1,1985,1437,R5YZ
2,1986,1181,R5WX
3,1986,1257,R5YZ
4,1987,1393,R5WX


In [24]:
r6_results = create_round_results(r6_seeds, 6, teams, tourney_slots_v2, results)
r6_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,Score_Strong,NumOT_Strong,Score_Weak,NumOT_Weak,TeamID_Winner
0,1985,R6CH,R5WX,R5YZ,6,1207,Georgetown,1437,Villanova,64,0,66,0,1437
1,1986,R6CH,R5WX,R5YZ,6,1181,Duke,1257,Louisville,69,0,72,0,1257
2,1987,R6CH,R5WX,R5YZ,6,1393,Syracuse,1231,Indiana,73,0,74,0,1231
3,1988,R6CH,R5WX,R5YZ,6,1242,Kansas,1328,Oklahoma,83,0,79,0,1242
4,1989,R6CH,R5WX,R5YZ,6,1371,Seton Hall,1276,Michigan,79,1,80,1,1276


## Combine results

In [25]:
comb_results = r1_results
for d in [r2_results, r3_results, r4_results, r5_results, r6_results]:
    comb_results = comb_results.append(d)
comb_results = comb_results.reset_index(drop=True)

In [27]:
comb_results.to_csv('../data/comb_results.csv', index=False)